# 🤖 심플한 Agent 생성 및 실행하기

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 12px; color: white; margin-bottom: 20px;">

### ✨ 첫 번째 에이전트 만들기

이 튜토리얼에서는 Azure OpenAI를 사용하여 간단한 AI 에이전트를 생성하고 실행하는 방법을 배웁니다.

</div>

## 1️⃣ 환경 변수 로드

<div style="background: #e8f5e9; border-left: 4px solid #4caf50; padding: 15px; border-radius: 8px; margin: 15px 0;">

📁 `.env` 파일에 설정된 Azure OpenAI 연결 정보를 로드합니다.

</div>

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # .env 파일 로드

---

## 2️⃣ 에이전트 생성

<div style="background: #e3f2fd; border-left: 4px solid #2196f3; padding: 15px; border-radius: 8px; margin: 15px 0;">

### 🎭 Joker 에이전트

Azure OpenAI를 사용하여 농담을 잘하는 에이전트를 생성합니다.

**주요 구성 요소:**
- 🔐 **인증**: Azure CLI 자격증명 사용
- 📝 **지시사항**: 한국어로 농담하는 어시스턴트
- 🏷️ **이름**: Joker

</div>

In [ ]:
import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="당신은 한국어로 말하는, 농담을 잘하는 어시스턴트입니다. 😄🎭",
    name="Joker"
)

<div style="background: #f5f5f5; border-left: 4px solid #9e9e9e; padding: 15px; border-radius: 8px; margin: 15px 0;">

### 📚 참고: 사용자 정의 환경 변수 사용

**출처**: [Microsoft Agent Framework Workshop](https://github.com/warnov/ms-agent-framework-step-by-step-workshop)

기본적으로 `.env` 파일의 **AZURE_OPENAI_ENDPOINT**와 **AZURE_OPENAI_CHAT_DEPLOYMENT_NAME**이 사용됩니다.

다른 환경 변수를 사용하려면 다음과 같이 명시적으로 지정할 수 있습니다:

```python
agent = AzureOpenAIChatClient(
    credential=AzureCliCredential(),
    endpoint=os.environ["AOAI_ENDPOINT"],
    deployment_name=os.environ["AOAI_DEPLOYMENT"]
).as_agent(
    instructions="당신은 이야기를 잘하는 어시스턴트입니다. 📖✨",
    name="Joker"
)
```

</div>

---

## 3️⃣ 기본 실행 방식

<div style="background: #fff3e0; border-left: 4px solid #ff9800; padding: 15px; border-radius: 8px; margin: 15px 0;">

### 🚀 에이전트 실행하기

에이전트의 `run()` 메서드를 호출하여 질문을 전달합니다.

**동작 방식:**
1. 📤 사용자 입력을 에이전트에 전달
2. 🤖 에이전트가 처리 후 응답 생성
3. 📥 `result.text`로 결과 확인

</div>

In [ ]:
async def main():
    result = await agent.run("해적에 관한 농담을 들려줘. 🏴‍☠️")
    print(result.text)

await main()

---

## 4️⃣ 스트리밍 실행 방식

<div style="background: #f3e5f5; border-left: 4px solid #9c27b0; padding: 15px; border-radius: 8px; margin: 15px 0;">

### 📡 실시간 스트리밍

`run_stream()` 메서드를 사용하면 응답을 실시간으로 받아볼 수 있습니다.

**장점:**
- ⚡ 빠른 첫 응답 시간
- 💬 실시간 대화 경험
- 🔄 점진적 결과 표시

**동작 방식:**
- 각 `update` 객체는 응답의 일부를 포함
- `update.text`로 텍스트 조각에 접근
- 모든 조각을 합쳐 완전한 응답 생성

</div>

In [ ]:
async def main():
    async for update in agent.run_stream("해적에 관한 긴 농담을 들려줘. 🏴‍☠️😄"):
        if update.text:
            print(update.text, end="", flush=True)
    print()  # 스트리밍 완료 후 새 줄 추가

await main()

---

## 5️⃣ 멀티모달: 이미지와 함께 질문하기

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 20px; border-radius: 12px; color: white; margin: 15px 0;">

### 🖼️ ChatMessage로 이미지 전달

텍스트뿐만 아니라 이미지도 함께 에이전트에 전달할 수 있습니다!

</div>

**방법 1: 온라인 이미지 URL 사용**

`ChatMessage` 객체를 생성하여 여러 유형의 콘텐츠를 조합할 수 있습니다:
- 📝 텍스트 (`Content.from_text()`)
- 🌐 온라인 이미지 (`Content.from_uri()`)

**구성 요소:**
| 항목 | 설명 |
|------|------|
| `role` | 메시지 역할 (USER, ASSISTANT, SYSTEM) |
| `contents` | 콘텐츠 배열 (텍스트, 이미지 등) |

In [ ]:
from agent_framework import ChatMessage, Content, Role

message = ChatMessage(
    role=Role.USER,
    contents=[
        Content.from_text(text="이 이미지에 대한 농담을 들려줘 😄🖼️"),
        Content.from_uri(uri="https://www.fotosanimales.es/wp-content/uploads/2017/12/pinguino.jpg", media_type="image/jpeg")
    ]
)

async def main():
    result = await agent.run(message)
    print(result.text)

await main()

**방법 2: 로컬 파일 시스템의 이미지 사용**

<div style="background: #e8f5e9; border-left: 4px solid #4caf50; padding: 15px; border-radius: 8px; margin: 15px 0;">

### 📁 로컬 이미지 불러오기

`Content.from_data()`를 사용하여 로컬 파일 시스템의 이미지를 에이전트에 전달할 수 있습니다.

**단계:**
1. 📂 이미지 파일을 바이너리 모드로 읽기
2. 🔄 `Content.from_data()`로 콘텐츠 생성
3. 📤 `ChatMessage`에 포함하여 전달

</div>

In [ ]:
from agent_framework import ChatMessage, Content, Role

# Load image from local file
with open("images/image1.jpg", "rb") as f:
    image_bytes = f.read()

message = ChatMessage(
    role=Role.USER,
    contents=[
        Content.from_text(text="이 이미지에서 무엇이 보이나요? 👀🖼️"),
        Content.from_data(
            data=image_bytes,
            media_type="image/jpeg"
        )
    ]
)

async def main():
    result = await agent.run(message)
    print(result.text)

await main()